正則表達式:TVBS清理作者格式方法

In [31]:
import re
#作者欄位清理方法，要放author[0]
def tvbs_author_etl(author_text):
    author = []
    author_text = re.sub(r"記者  ", "", author_text)
    author_text = re.sub(r" / 攝影.*", "", author_text)
    author_text = re.sub(r" / ", "", author_text)
    author_text = re.sub(r"編輯  ", "", author_text)
    author_text = re.sub(r" 報導", "", author_text)
    author_text = re.sub(r" ", ",", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find(",") != -1:
        author_text = author_text.split(",")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

正則表達式:SETN清理作者格式方法

In [32]:
#SETN作者欄位清理方法，要放author[0]
def setn_author_etl(author_text):
    author = []
    #將／XX報導XX字串清除
    author_text = re.sub(r"／.*報導.*", "", author_text)
    #將／XX特稿XX字串清除
    author_text = re.sub(r"／.*特稿.*", "", author_text)
    #將記者字串清除
    author_text = re.sub(r"記者", "", author_text)
    #將文／字串清除
    author_text = re.sub(r"文／", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    else:
        author.append(author_text)
    return author

正則表達式:ETtoday清理作者格式方法

In [49]:
#ETtoday作者欄位清理方法，要放author[0]
def ettoday_author_etl(author_text):
    author = []
    author_text = re.sub("／.?.?報導.*", "", author_text)
    author_text = re.sub(".*記者", "", author_text)
    author_text = re.sub("\xa0", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

文章正反面情緒分析 需要的套件

In [33]:
#在dockerfile的指令
#RUN !pip install udicOpenData
#RUN !pip install Swinger
#ADD ./dict_all.txt /opt/conda/lib/python3.6/site-packages/udicOpenData/dictionary/
from udicOpenData.dictionary import *
jieba.load_userdict(os.path.join(DIR_NAME, 'dict_all.txt'))
#要等一下子
from Swinger import Swinger

文章正反面情緒分析方法

In [34]:
def emo_swinger(content, model='LogisticRegression'):
    s = Swinger()
    s.load(model) 
    # default model= "LogisticRegression", 可替換為"MultinomialNB"
    emo_result = s.swing(content)
    if emo_result == "pos":
        result = "positive"
    elif emo_result == "neg":
        result = "negative"
    return result

文章藍綠貼標分析 需要的套件

In [35]:
!pip install pandas 
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

將jieba斷詞寫成方法

In [36]:
def cutflow(p):
    cutresult = " ".join(jieba.cut(p))
    return cutresult.replace("\r", "").replace("\n", "")

載入詞向量模型與貝氏模型

In [37]:
clg = joblib.load('clg') 
vec = joblib.load('vec') 

文章藍綠貼標分析方法:貝氏定理

In [38]:
def label_predict(text):
    docs_news = cutflow(text)
    c=[{"content" : docs_news,
       "tag" : 0}]
    d = pd.DataFrame(c)
    test_counts = vec.transform(d["content"])
    pre = clg.predict(test_counts)
    label_result = list(pre)[0]
    if label_result == '1':
        result = "blue"
    elif label_result == '0':
        result = "green"
    return result

文章摘要分析 需要的套件

In [39]:
!pip install networkx
!pip install textrank4zh
from  __future__  import print_function
import jieba
import numpy
import networkx
import sys
import codecs 
from textrank4zh import TextRank4Keyword ,TextRank4Sentence

文章摘要方法:TextRank

In [40]:
def news_summary(text):
    tr4s = TextRank4Sentence()
    tr4s.analyze( text = text, lower = True , source = ' all_filters ' )
    for item in tr4s.get_key_sentences( num = 1 ):
        return item.sentence

Elasticsearch 需要的套件

In [41]:
!pip install elasticsearch
from elasticsearch import Elasticsearch

In [42]:
# 建立Elasticsearch連線
es = Elasticsearch(['elasticsearch:9200'])

In [ ]:
# 在Elasticsearch建立index，亦可加入body參數先定義mapping格式
es.indices.create(index = 'news', ignore = 400)

In [ ]:
# 將Elasticsearch定義所放入的資料格式:mapping
es.indices.put_mapping(index = "news",
                       doc_type = "politics",                
                       body = {
                        "properties": {
                            "source": {"type": "text"},
                            "url": {"type": "text"},
                            "title": {"type": "text"},
                            "date_": {"type": "text"},
                            "author": {"type": "text"},
                            "content": {"type": "text"},
                            "kw": {"type": "text"},
                            "img_url": {"type": "text"},
                            "@timestamp": {"type": "date"}
                        }
                    }
                )

In [43]:
# 為了將匯入資料的時間加入時區
from datetime import datetime, timedelta, timezone
tz_utc_8 = timezone(timedelta(hours=8)) # 創建時區UTC+8:00

Kafka Consumer需要的套件

In [44]:
!pip install kafka
from kafka import KafkaConsumer, TopicPartition
import sys
import json, time, requests

In [ ]:
if __name__ == "__main__":
    # 步驟1.設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # Kafka集群在那裡?
        bootstrap_servers=["kafka1:29092"],
        # ConsumerGroup的名稱
        group_id="test01",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
        key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        #value_deserializer=bytes.decode,
        value_deserializer=lambda m: json.loads(m.decode('utf-8')),
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )
    # 步驟2.指定想要訂閱訊息的topic名稱
    topic_name = "test"
    # 步驟3.讓Consumer向Kafka集群訂閱指定的topic
    consumer.subscribe(topics = topic_name)


    # 步驟4.持續的拉取Kafka有進來的訊息
    try:
        print("Start listen incoming messages ...","\n")
        # 持續監控是否有新的record進來 - 方法一
        for record in consumer:
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
        
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            
            # 將新聞作者欄位做處理後再放回
            if msgValue["source"] == "TVBS":      
                msgValue["author"] = tvbs_author_etl(msgValue["author"][0])
            elif msgValue["source"] == "SETN":
                msgValue["author"] = setn_author_etl(msgValue["author"][0])
            elif msgValue["source"] == "ETtoday":
                msgValue["author"] = ettoday_author_etl(msgValue["author"][0])
            
            # 將新聞內文做文章情緒分析並加入欄位與結果
            msgValue["ariticle_emotion"] = emo_swinger(msgValue["content"])
            # 將新聞內文做藍綠分析並加入欄位結果
            msgValue["label"] = label_predict(msgValue["content"])
            # 將新聞做文章摘要並加入欄位與結果
            msgValue["abstract"] = news_summary(msgValue["content"])
            
            # 秀出metadata與msgKey & msgValue訊息
            print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition,
                                                                              record.offset, record.key, msgValue))
            print("\n")
            
            print("Start sending data to MySQL !!!")
            # 將資料送入MySQL
            ip_location = 'chatbot_api'
                         
            News = msgValue
            
            # 將json傳回API Server
            Endpoint = 'http://%s:5000/news' % (ip_location)

            # header要特別註明是json格式
            Header = {'Content-Type':'application/json'}
            
            # 傳送post對API server新增資料 
            Response = requests.post(Endpoint,headers=Header,data=json.dumps(News))

            # 印出Response的資料訊息
            print(Response)
            Response = Response.json()
            print(Response)
            print("\n")
            
        
            print("Start sending data to elasticsearch !!!")
            # 將資料送入Elasticsearch
            elasticsearch_data = msgValue
            elasticsearch_data["@timestamp"] = datetime.now().replace(tzinfo = tz_utc_8)            
            # 將資料放入Elasticsearch，若沒有index，會直接建立index、type 
            es.index(index = 'news', doc_type = 'politics', body = elasticsearch_data, refresh = True )
            print("\n")

    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        consumer.close()

Start listen incoming messages ... 

load default bestMainFeatures
load default bestMainFeatures success!!
load model from LogisticRegression


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.18.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.18.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


topic=test, partition=0, offset=0 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181224/1339321.htm', 'title': '立法院前秘書長林錫山涉貪 監察院9比0全數通過彈劾案', 'date_': '2018/12/24 17:07', 'author': ['政治中心'], 'content': '立法院前秘書長林錫山被控收賄案，涉嫌索賄3950萬元，並有2.3億不明財產，二審遭判15年、褫奪公權5年，沒收犯罪所得2.6億餘元。監察院24日以9比0的票數通過林錫山彈劾案。監察院指出，林錫山於任職期間，假借權力，收受賄賂，以圖本身之利益，涉犯不違背職務收受賄賂罪、公務員財產來源不明罪及洗錢罪，且未依法據實申報財產，有違公務員服務法之規定，其違法失職事證明確，情節重大，依法提案彈劾。本案發生於2016年初，林錫山遭檢調查出涉嫌利用辦理採購電腦軟、硬體招標案的機會，收受廠商網遠科技負責人李保承3950萬元的賄賂，甚至在高鐵上收受賄賂金錢，檢方發動搜索立法院，並對林錫山聲請羈押獲准，全案於同年5月偵查終結，林錫山等13人遭提起公訴。一審法院去年5月間宣判，重判林錫山有期徒刑16年、褫奪公權6年，另外貪污所得3950萬元、不明財產2.3億餘元，均要追繳沒收抵償。案經上訴，高等法院審理期間，林錫山坦承犯下職務上行為收賄罪，但稱自己遭廠商蠱惑，「我把持不住」。至於不明財產罪部分，他辯稱父親留有遺產，再加上自己的投資，「不能因為我有錢，就說是財產來源不明，將財產全都沒收。」不過林在高院審理期間，繳交貪污所得2800萬元；至於財產來源不明的2.4億餘元，他除了繳回部分現金以外，還陸續提出名下不動產擔保，總計提出2.18億餘元的財物。林錫山並以照顧老母親為由，在過年前請求交保。高院農曆年前作出裁定，准許林以5000萬元交保，並限制住居於臺北市大安區住所，且限制出境、出海，另要求林應於每星期一、三、五，至臺北市政府警察局大安分局安和路派出所報到。至於本案部分，高院29日做出二審宣判，改認定林錫山犯下8個《貪污治罪條例》中的「職務上行為收賄罪」、1個「財產來源不明罪」、2個「洗錢罪」，合併執行有期徒刑15年，褫奪公權

topic=test, partition=0, offset=6 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181224/1339177.htm', 'title': '館長笑不敢同台直播！卓榮泰不怕：時間很快會敲定', 'date_': '2018/12/24 14:58', 'author': ['政治中心'], 'content': '網路紅人「館長」陳之漢日前聽聞行政院祕書長卓榮泰要約他直播，大嗆「「我笑你不敢來」，並表示沒有收到任何的邀請。對此，卓榮泰24日受訪時回應，已經開始邀約了，時堅很快就會敲定。據《鏡週刊》報導，卓榮泰認為，想要聆聽支持者的聲音，「我可以去跟陳其邁喝咖啡，我也可以去跟館長聊一聊啊！我可以唱詹雅雯的〈深情海岸〉，我也可以找鄭進一來唱〈家後〉吧！」館長表示，沒收到卓榮泰的聯絡，自己很想跟一個要做黨主席的人一起直播，「我第一個就問促轉會」、「我很想問你，你當黨主席的時候，你的立委能不能通過一些有意義的審查案」、「我笑你不敢來！你要來不用新聞叫囂、直接打電話來嘛！」針對館長在直播中的嗆聲，卓榮泰說，可以和所有對民進黨有理性對話的人談話，而和館長同台直播已經在敲定時間了，時程表很快就會出來；而是否擔心館長問題太過心辣，卓榮泰說，當然會，不過會努力讓大家了解未來民進黨的轉變。卓榮泰參選黨主席後，將和館長一起直播，外界也好奇，未來是否有可能和其他網紅對話？他表示，不排斥任何可能性，只要願意理性對話者，他都可以趁機把民進黨未來改變的方式、方向和各界交換意見，自己都非常非常樂意。', 'kw': ['卓榮泰', '館長', '直播', '民進黨'], 'img_url': 'https://cdn2.ettoday.net/images/3785/d3785867.jpg', 'ariticle_emotion': 'positive', 'label': 'green', 'abstract': '」館長表示，沒收到卓榮泰的聯絡，自己很想跟一個要做黨主席的人一起直播，「我第一個就問促轉會」、「我很想問你，你當黨主席的時候，你的立委能不能通過一些有意義的審查案」、「我笑你不敢來'})


Start sending data 